# Домашнее задание: BiLSTM для задачи PoS Tagging

В этом ноутбуке мы будем создавать модель машинного обучения, которая генерирует результат для каждого элемента входной последовательности с использованием PyTorch и TorchText. Конкретно, мы будем подавать текст на вход, а модель будет выводить метку - часть речи (PoS) для каждого токена во входном тексте. Этот подход также может применяться для распознавания именованных сущностей (NER), где результатом для каждого токена будет указание на тип сущности, если таковая имеется.

В этом блокноте мы реализуем многослойную двунаправленную LSTM (BiLSTM) для предсказания меток частей речи с использованием набора данных Universal Dependencies English Web Treebank (UDPOS).

In [1]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random

Зафиксируем случайности для воспроизводимости результатов.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

В этом наборе данных есть два разных набора меток: метки универсальных зависимостей (UD) и метки Penn Treebank (PTB). Мы будем обучать модель только на метках UD, но загрузим метки PTB, чтобы показать, как их можно использовать вместо них.

* UD_TAGS определяет, как следует обрабатывать метки UD. В нашем словаре TEXT, который мы создадим позже, будут неизвестные токены, то есть токены, которых нет в нашем словаре. Однако у нас не будет неизвестных меток, поскольку мы имеем дело с конечным набором возможных меток. Мы будем обозначать неизвестные токены как <unk>, и затем будем их убирать, установив unk_token = None.

* PTB_TAGS выполняет то же самое, что и UD_TAGS, но обрабатывает метки PTB.

In [4]:
from torchtext.data import Field

TEXT = Field(lower = True)
UD_TAGS = Field(unk_token = None)
PTB_TAGS = Field(unk_token = None)

In [5]:
fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

Загрузим датасет UDPOS.

In [7]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:00<00:00, 2.16MB/s]


extracting


## Задание

Посмотрите на количество объектов в датасетах `train_data, valid_data и test_data`. В ответ запишите число объектов в самом маленьком датасете.

In [14]:
len(train_data), len(valid_data), len(test_data)

(12543, 2002, 2077)

In [13]:
for el in train_data[:1]:
  print(el.text, el.udtags)

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.'] ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


Напечатаем пример из датасета

In [15]:
print(vars(train_data.examples[0]))

{'text': ['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.'], 'udtags': ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], 'ptbtags': ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']}


Можем отдельно посмотреть на текст и на теги

In [ ]:
print(vars(train_data.examples[0])['text'])

In [ ]:
print(vars(train_data.examples[0])['udtags'])

In [ ]:
print(vars(train_data.examples[0])['ptbtags'])

Что мы сделаем дальше:

* Мы создадим словарь - отображение токенов в целые числа.

* Мы хотим, чтобы в нашем наборе данных были некоторые неизвестные токены, чтобы воссоздать, как эта модель будет использоваться в реальной жизни, поэтому мы устанавливаем `min_freq = 2`, что означает, что в словарь будут добавлены только токены, появляющиеся хотя бы дважды в обучающем наборе, и остальные будут заменены токенами `<unk>`.

* Мы также загружаем предобученные векторы GloVe длины 100 для инициализации эмбеддингов.

* `unk_init` используется для инициализации эмбеддингов токенов, которых нет в словаре предварительно обученных вложений. По умолчанию эта инициализация устанавливает эти эмбеддинги в нули, однако лучше избежать их инициализации одним и тем же значением, поэтому мы инициализируем их из нормального распределения.

* Предобученные векторы загружаем в наш словарь и будем инициализировать нашу модель этими значениями позже.

## Задание

По тренировочным данным постройте три словаря, используя `build_vocab`:

* Cловарь по текстам `TEXT` с гиперпараметрами:
  * min_freq = MIN_FREQ
  * vectors = "glove.6B.100d"
  * unk_init = torch.Tensor.normal_

* Словарь по `UD_TAGS`

* Словарь по `PTB_TAGS`

Сколько уникальных токенов в словаре, построенном по текстам?

In [17]:
MIN_FREQ = 2

# build_vocab -- создать словарь по данному полю в датасете
TEXT.build_vocab(
    train_data,
    min_freq=MIN_FREQ,
    vectors="glove.6B.100d",
    unk_init = torch.Tensor.normal_
)

# В тэгах/лейблах не используем гиперпараметры, потому как там просто метки, эта обработка бессмысленна
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:28<00:00, 13908.94it/s]


In [20]:
print(f'{len(TEXT.vocab)=}, {len(UD_TAGS.vocab)=}, {len(PTB_TAGS.vocab)=}')

len(TEXT.vocab)=8866, len(UD_TAGS.vocab)=18, len(PTB_TAGS.vocab)=51


## Задание

Какой самый популярный (часто встречающийся) токен в словаре, построенном по текстам?

In [22]:
TEXT.vocab.freqs.most_common(5)

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002)]

Посмотрим на функцию, вычисляющую процентное соотношение тегов в текстах.

In [23]:
def tag_percentage(tag_counts):

    total_count = sum([count for tag, count in tag_counts])

    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

## Задание

Пользуясь функцией `tag_percentage`, выведите на экран процентное соотношение каждого UD-тэга.

Какой тег встречается в текстах чаще всего (в процентах)?

In [28]:
UD_TAGS.vocab.freqs.most_common()

[('NOUN', 34781),
 ('PUNCT', 23679),
 ('VERB', 23081),
 ('PRON', 18577),
 ('ADP', 17638),
 ('DET', 16285),
 ('PROPN', 12946),
 ('ADJ', 12477),
 ('AUX', 12343),
 ('ADV', 10548),
 ('CCONJ', 6707),
 ('PART', 5567),
 ('NUM', 3999),
 ('SCONJ', 3843),
 ('X', 847),
 ('INTJ', 688),
 ('SYM', 599)]

In [29]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

NOUN		34781		17.0%
PUNCT		23679		11.6%
VERB		23081		11.3%
PRON		18577		 9.1%
ADP		17638		 8.6%
DET		16285		 8.0%
PROPN		12946		 6.3%
ADJ		12477		 6.1%
AUX		12343		 6.0%
ADV		10548		 5.2%
CCONJ		6707		 3.3%
PART		5567		 2.7%
NUM		3999		 2.0%
SCONJ		3843		 1.9%
X		847		 0.4%
INTJ		688		 0.3%
SYM		599		 0.3%


## Задание

Используя `BucketIterator.split`, создайте объекты `train_iterator, valid_iterator, test_iterator` для итерирования по батчам.

In [30]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [33]:
for item in train_iterator:
    print(item.text, item.udtags)
    print(item.text.shape, item.udtags.shape)
    break

tensor([[ 352, 6609,   14,  ...,  381,  132,   70],
        [ 545,    1,   29,  ...,  121,   14,  632],
        [ 144,    1,   67,  ..., 3042,   11,  500],
        ...,
        [   1,    1,    1,  ...,    0,    1,    1],
        [   1,    1,    1,  ...,   31,    1,    1],
        [   1,    1,    1,  ...,    3,    1,    1]], device='cuda:0') tensor([[ 8,  7,  4,  ...,  3, 10,  6],
        [ 1,  0,  9,  ...,  5,  4,  1],
        [ 9,  0, 10,  ...,  1,  9,  1],
        ...,
        [ 0,  0,  0,  ...,  1,  0,  0],
        [ 0,  0,  0,  ...,  2,  0,  0],
        [ 0,  0,  0,  ...,  2,  0,  0]], device='cuda:0')
torch.Size([52, 128]) torch.Size([52, 128])


## Создаем архитектуру нейронной сети

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bidirectional-lstm.png?raw=1)

Задайте нейронную сеть по аналогии с сетью из вебинара:

* Слой Embedding:
  * помимо прочего задайте `padding_idx = pad_idx`

* Затем слой LSTM с гиперпараметрами:
  * `n_layers = 1`
  * `bidirectional = True`
  * задайте `dropout`

* Затем DropOut слой

* Линейный слой, принимающий на вход `hidden_dim * 2` нейронов (так как двунаправленная сеть) и на выходе `output_dim` нейронов

В ответ на задание выберите, как выглядит первая строчка в архитектуре сети.

In [81]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim=embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True, num_layers=1, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)


    def forward(self, text):

        #text = [sent len, batch size]
        #pass text through embedding layer and then through dropout layer
        embedded = self.dropout(self.embedding(text))

        #pass embeddings into LSTM

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        #outputs содержит прямые и обратные скрытые состояния в конечном слое
        #hidden и cell - состояния скрытого слоя и ячейки в обратном и прямом направлении на последнем временном шаге

        outputs, (hidden, cell) = self.lstm(embedded)

        #apply dropout and then linear layer
        predictions = self.fc(self.dropout(outputs))

        return predictions

## Обучение модели

## Задание

Запустите ячейку ниже. Если класс `BiLSTMPOSTagger` реализован корректно, ячейка отработает без ошибок. Получилось?

In [82]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM,
                        EMBEDDING_DIM,
                        HIDDEN_DIM,
                        OUTPUT_DIM,
                        N_LAYERS,
                        BIDIRECTIONAL,
                        DROPOUT,
                        PAD_IDX)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Инициализируем веса сети числами из стандартного нормального распределения

In [69]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(8866, 100, padding_idx=1)
  (lstm): LSTM(100, 128, dropout=0.25, bidirectional=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=256, out_features=18, bias=True)
)

In [45]:
model.parameters

<bound method Module.parameters of BiLSTMPOSTagger(
  (embedding): Embedding(8866, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=256, out_features=18, bias=True)
)>

In [50]:
for el in model.parameters():
  print(el.numel())

886600
51200
65536
512
512
51200
65536
512
512
131072
65536
512
512
131072
65536
512
512
4608
18


## Задание

Напишите функцию для вычисления количества весов сети.

С помощью этой функции выведите на экран число весов нашей сети.

In [51]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Модель содержит {count_parameters(model):,} обучаемых параметров')

Модель содержит 1,522,010 обучаемых параметров


## Задание

Инициализируйте embedding-слой сети предобученными GloVe-векторами.

В ответ напишите число координат в предобученных эмбеддингах.

In [83]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([8866, 100])


In [84]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9261,  2.3049,  0.5502,  ..., -0.3492, -0.5298, -0.1577],
        [-0.5972,  0.0471, -0.2406,  ..., -0.9446, -0.1126, -0.2260],
        [-0.4809,  2.5629,  0.9530,  ...,  0.5278, -0.4588,  0.7294]])

Инициализируем нулями pad-токены

In [54]:
EMBEDDING_DIM

100

In [57]:
PAD_IDX

1

In [85]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.9261,  2.3049,  0.5502,  ..., -0.3492, -0.5298, -0.1577],
        [-0.5972,  0.0471, -0.2406,  ..., -0.9446, -0.1126, -0.2260],
        [-0.4809,  2.5629,  0.9530,  ...,  0.5278, -0.4588,  0.7294]])


Зададим оптимизатор

In [86]:
optimizer = optim.Adam(model.parameters())

Зададим loss.

В случае токена `<pad>` (пустота) лосс мы не считаем, поэтому индексы таких токенов мы пропускаем (игнорируем).

In [87]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [59]:
TAG_PAD_IDX

0

Переносим модель на GPU по возможности

In [88]:
model = model.to(device)
criterion = criterion.to(device)

Функция ниже вычисляет `accuracy` для каждого батча

In [89]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

## Задание

Допишите цикл обучения модели.

Для каждого батча на каждой итерации:
- зануляем градиенты
- применяем модель к батчу
- делаем reshape прогнозов, так как loss нельзя вычислить для тензора размерности 3 (это уже написано)
- вычисляем loss и accuracy
- вычисляем градиенты и делаем шаг градиентного спуска

In [98]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

      text = batch.text
      tags = batch.udtags
      optimizer.zero_grad()

      #text = [sent len, batch size]

      predictions = model(text)
      #predictions = [sent len, batch size, output dim]
      #tags = [sent len, batch size]

      predictions = predictions.view(-1, predictions.shape[-1]) # predictions - прогнозы модели
      tags = tags.view(-1) # tags - правильные ответы (метки)

      loss = criterion(predictions, tags)
      acc = categorical_accuracy(predictions, tags, tag_pad_idx)
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()
      epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Функцию `evaluate` для простоты мы написали.

In [95]:
def evaluate(model, iterator, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Ниже функция, которая замеряет время обучения на каждой эпохе

In [96]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Задание

Обучим нашу модель. Допишите цикл по подсказкам в коде.

Какая accuracy (в процентах) получается на валидации на последней эпохе? Ответ округлите до целого числа.

In [99]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX) # примените функцию для обучения модели
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX) # ваш код здесь - примените функцию для применения и оценки качества модели

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time) # замерьте время выполнения эпохи, используя написанную для этого функцию

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')


    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 2s
	Train Loss: 1.525 | Train Acc: 55.83%
	 Val. Loss: 0.834 |  Val. Acc: 76.60%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.508 | Train Acc: 84.93%
	 Val. Loss: 0.559 |  Val. Acc: 82.71%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.356 | Train Acc: 89.13%
	 Val. Loss: 0.480 |  Val. Acc: 84.62%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.293 | Train Acc: 90.88%
	 Val. Loss: 0.448 |  Val. Acc: 85.49%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.258 | Train Acc: 91.89%
	 Val. Loss: 0.422 |  Val. Acc: 86.19%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.233 | Train Acc: 92.65%
	 Val. Loss: 0.410 |  Val. Acc: 86.51%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.214 | Train Acc: 93.16%
	 Val. Loss: 0.395 |  Val. Acc: 87.26%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.199 | Train Acc: 93.69%
	 Val. Loss: 0.389 |  Val. Acc: 87.01%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.185 | Train Acc: 94.06%
	 Val. Loss: 0.378 |  Val. Acc: 88.91%
Epoch: 10 | Epoch Time: 0m 1

Посмотрим на качество обученной модели на тесте

In [100]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.395 |  Test Acc: 88.67%


## Инференс

Посмотрим, как модель работает на новых данных. Допишите функцию `tag_sentence` для применения обученной модели, по подсказкам ниже.

In [101]:
def tag_sentence(model, device, sentence, text_field, tag_field):

    model.eval() # ваш код здесь - переведите модель в режим применения

    if isinstance(sentence, str):
        nlp = spacy.load('en_core_web_sm')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]

    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens] # ваш код здесь - создайте список, состоящий из переведенных в индексы токенов из словаря text_field.vocab (используйте stoi)

    unk_idx = text_field.vocab.stoi[text_field.unk_token]

    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]

    token_tensor = torch.LongTensor(numericalized_tokens) # ваш код здесь - приведите numericalized_tokens к типу torch.LongTensor

    token_tensor = token_tensor.unsqueeze(-1).to(device)

    predictions = model(token_tensor) # ваш код здесь - примените модель к token_tenzor

    top_predictions = predictions.argmax(-1)

    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]

    return tokens, predicted_tags, unks

## Задание

Запустите две следующие ячейки. Проверим, что написанная функция работает корректно.

В ответе выберите те токены, которые были нераспознаны (их не было в обучающих данных).

In [102]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['udtags']

print(sentence)

['[', 'this', 'killing', 'of', 'a', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', '.', ']']


In [103]:
tokens, pred_tags, unks = tag_sentence(model,
                                       device,
                                       sentence,
                                       TEXT,
                                       UD_TAGS)

print(unks)

['respected', 'cleric']


## Задание

Проверим качество модели. Запустите ячейку ниже. В ответе укажите число неверно классифицированных токенов.

In [104]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

PUNCT		PUNCT		✔		[
DET		DET		✔		this
VERB		NOUN		✘		killing
ADP		ADP		✔		of
DET		DET		✔		a
NOUN		ADJ		✘		respected
NOUN		NOUN		✔		cleric
AUX		AUX		✔		will
AUX		AUX		✔		be
VERB		VERB		✔		causing
PRON		PRON		✔		us
NOUN		NOUN		✔		trouble
ADP		ADP		✔		for
NOUN		NOUN		✔		years
PART		PART		✔		to
VERB		VERB		✔		come
PUNCT		PUNCT		✔		.
PUNCT		PUNCT		✔		]


## Задание

Примените модель к любому предложению (на английском языке). Какая доля токенов размечена верно? Ответ напишите в комментариях.

In [106]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, tags):
    print(f"{tag}\t\t{token}")

Pred. Tag	Token

PRON		i
VERB		love
DET		this
NOUN		game
PUNCT		.


In [105]:
sentence = 'I love this game.'

tokens, tags, unks = tag_sentence(model,
                                  device,
                                  sentence,
                                  TEXT,
                                  UD_TAGS)



[]


In [107]:
for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

PUNCT		PUNCT		✔		i
DET		DET		✔		love
VERB		NOUN		✘		this
ADP		ADP		✔		game
DET		DET		✔		.
